In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Function to import and merge all pickled files with a common pattern
def import_and_merge_pickles(directory, common_pattern):
    merged_df = pd.DataFrame()
    for file in os.listdir(directory):
        if common_pattern in file and file.endswith('.pkl'):
            file_path = os.path.join(directory, file)
            temp_df = pd.read_pickle(file_path)
            merged_df = pd.concat([merged_df, temp_df], ignore_index=True)
    return merged_df

# Specify directory and common pattern
directory = '/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data'
common_pattern = 'bibliometric_database_20241013_tiab'

# Import and merge all matching pickled files
merged_df = import_and_merge_pickles(directory, common_pattern)

# Convert 'OpenAlex Publication Year' to numeric and handle any errors
merged_df['publication_year'] = pd.to_numeric(merged_df['OpenAlex Publication Year'], errors='coerce')

df = merged_df.copy()
df

,OpenAlex DOI,OpenAlex ID,OpenAlex Title,OpenAlex Abstract,OpenAlex Publication Year,OpenAlex Publication Date,OpenAlex Authors,OpenAlex First Authors,OpenAlex Last Authors,OpenAlex Corresponding Authors,OpenAlex Institutions,OpenAlex Institutions Type,OpenAlex First Authors Institutions Countries,OpenAlex Last Authors Institutions Countries,OpenAlex Institutions Distinct Count,OpenAlex Countries,OpenAlex First Authors Countries,OpenAlex Last Authors Countries,OpenAlex Corresponding Authors Countries,OpenAlex Countries Distinct Count,OpenAlex Journal,is_in_doaj,OpenAlex Crossref Publisher,OpenAlex Crossref Subject,OpenAlex topics,OpenAlex Keywords,OpenAlex Mesh,OpenAlex Concepts,OpenAlex Sustainable Development Goals,Crossref Citation count,OpenAlex Cited by Count,OpenAlex Counts by Year,OpenAlex Publication Type,OpenAlex Indexed In,OpenAlex Language,OpenAlex Open Access,OpenAlex Open Access status,OpenAlex grants,Crossref Funders,Altmetric Score,Altmetric Read Count,Altmetric Image,Altmetric URL,apc_list_value,apc_list_currency,apc_list_value_usd,apc_list_provenance,apc_paid_value,apc_paid_currency,apc_paid_value_usd,apc_paid_provenance,number_of_authors,Citations 2024,Citations 2023,Citations 2022,Citations 2021,Citations 2020,Citations 2019,Citations 2018,Citations 2017,Citations 2016,Citations 2015,Citations 2014,Citations 2013,Citations 2012,OpenAlex Countries Full Name,OpenAlex Countries Region,OpenAlex Countries Income,OpenAlex First Authors Countries Full Name,OpenAlex First Authors Countries Region,OpenAlex First Authors Countries Income,OpenAlex Last Authors Countries Full Name,OpenAlex Last Authors Countries Region,OpenAlex Last Authors Countries Income,OpenAlex Corresponding Authors Countries Full Name,OpenAlex Corresponding Authors Countries Region,OpenAlex Corresponding Authors Countries Income,publication_year
0,https://doi.org/10.1007/s11192-009-0146-3,https://openalex.org/W2150220236,"Software survey: VOSviewer, a computer program...","We present VOSviewer, a freely available compu...",2009,2009-12-30,"Nees Jan van Eck, Ludo Waltman",Nees Jan van Eck,Ludo Waltman,,Leiden University (NL),Leiden University (education),Leiden University (NL),Leiden University (NL),1,NL,NL,NL,,1,Scientometrics,False,Springer Science and Business Media LLC,,Information Visualization and Visual Data Mini...,"Information Visualization, Interactive Visuali...",,"Computer science, Construct (python library), ...",,9908,12322,"[{'year': 2024, 'cited_by_count': 2487}, {'yea...",article,"['crossref', 'pubmed']",en,True,hybrid,,,87.484,9223.0,https://badges.altmetric.com/?size=64&score=88...,https://www.altmetric.com/details.php?citation...,2290.0,EUR,2890.0,doaj,2290.0,EUR,2890.0,doaj,2,2487.0,3183.0,2441.0,1652.0,991.0,571.0,359.0,172.0,132.0,94.0,78.0,44.0,27.0,Netherlands,Europe,High income,Netherlands,Europe,High income,Netherlands,Europe,High income,,,,2009
1,https://doi.org/10.1177/1094428114562629,https://openalex.org/W3125707221,Bibliometric Methods in Management and Organiz...,We aim to develop a meaningful single-source r...,2014,2014-12-22,"Ivan Župič, Tomaž Čater",Ivan Župič,Tomaž Čater,Ivan Župič,University of Ljubljana (SI),University of Ljubljana (education),University of Ljubljana (SI),University of Ljubljana (SI),1,SI,SI,SI,SI,1,Organizational Research Methods,False,SAGE Publications,,Knowledge Management and Organizational Innova...,"Bibliographic coupling, Bibliometric Analysis,...",,"Bibliographic coupling, Citation, Bibliometric...",Quality education,3038,3985,"[{'year': 2024, 'cited_by_count': 806}, {'year...",article,['crossref'],en,True,green,,,19.600,4730.0,https://badges.altmetric.com/?size=64&score=20...,https://www.altmetric.com/details.php?citation...,NaN,None,NaN,None,NaN,None,NaN,None,2,806.0,1099.0,861.0,544.0,339.0,169.0,78.0,38.0,19.0,1.0,2.0,1.0,NaN,Slovenia,Europe,High income,Slovenia,Europe,High income,Slovenia,Europe,High income,Slovenia,Europe,High income,2014
2,https://doi.org/10.1016/j.jbusres.2021

In [ ]:
#df = pd.read_pickle('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_20241011.pkl')
pd.set_option('display.max_columns', None)
print(len(df))
#df.head()

71468


In [ ]:
import pandas as pd
import re
import numpy as np

# Load data
# df = pd.read_json('openalex_data.json')  # You can use pd.read_csv() if your data is in CSV format

# Define function to clean text
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        return text.lower()
    return ""

# Clean and lowercase title, abstract, and keywords
df['cleaned_title'] = df['OpenAlex Title'].apply(clean_text)
df['cleaned_abstract'] = df['OpenAlex Abstract'].apply(clean_text)
df['cleaned_keywords'] = df['OpenAlex Keywords'].apply(clean_text)

# List of literature databases with main names and synonyms grouped
literature_databases_mapping = {
    'Internet Archive Scholar': ['internet archive scholar'],
    'CORE': ['connecting repositories', 'core.ac.uk'],
    'CiteSeerX': ['citeseerx', 'citeseer'],
    'Paperity': ['paperity'],
    'Semantic Scholar': ['semantic scholar'],
    'Europe PMC': ['europe pmc'],
    'PubMed Central (PMC)': ['pubmed central', 'pmc'],
    'ResearchGate': ['researchgate', 'research gate'],
    'SSRN': ['ssrn', 'social science research network', 'ssrn elibrary'],
    'HAL': ['hal'],
    'RePEc': ['repec', 'econpapers', 'research papers in economics'],
    'PhilPapers': ['philpapers'],
    'ERIC': ['eric', 'educational resource information center'],
    'Synthical': ['synthical'],
    'Zendy': ['zendy'],
    'AGRIS': ['agris', 'agricultural database'],
    'Arachne': ['arachne'],
    'AMiner': ['aminer'],
    #'Arts & Humanities Citation Index': ['arts & humanities citation index', 'ahci', 'web of science ahci'],
    'Astrophysics Data System': ['astrophysics data system', 'ads', 'nasa ads'],
    'ATLA Religion Database': ['atla religion database'],
    'BASE': ['bielefeld academic search engine'], #'base',
    'Book Review Index Online': ['book review index online'],
    'Books in Print': ['books in print'],
    'CAB Abstracts': ['cab abstracts', 'cabi'],
    'CINAHL': ['cinahl', 'cumulative index to nursing and allied health'],
    'CiNii': ['cinii'],
    'CNKI': ['cnki', 'cnki scholar'],
    'Crossref': ['crossref'],
    'DeepDyve': ['deepdyve'],
    'Dimensions': ['dimensions database', 'dimensions.ai', 'digital science dimensions'],
    'Directory of Open Access Journals (DOAJ)': ['doaj', 'directory of open access journals'],
    'DBLP': ['dblp', 'computer science bibliography'],
    'EconBiz': ['econbiz'],
    'EconLit': ['econlit', 'aea econlit'],
    'EMBASE': ['embase'],
    'FSTA': ['fsta', 'food science and technology abstracts'],
    'GeoRef': ['georef'],
    'Google Scholar': ['google scholar', 'scholar google'],
    'Informit': ['informit'],
    'Inspec': ['inspec', 'iet inspec'],
    'International Nuclear Information System (INIS)': ['inis', 'international nuclear information system'],
    'International Philosophical Bibliography': ['international philosophical bibliography'],
    'J-Gate': ['j-gate', 'informatics j-gate'],
    'The Lens': ['the lens', 'patent lens', 'lens.org'],
    'MathSciNet': ['mathscinet', 'ams mathscinet'],
    #'MEDLINE': ['medline', 'pubmed'],
    'MyScienceWork': ['mysciencework'],
    'National Diet Library Collection': ['national diet library collection'],
    'OAIster': ['oaister'],
    'OpenAIRE Graph': ['openaire graph', 'openaire'],
    'OpenAlex': ['openalex'],
    'PsycINFO': ['psycinfo', 'apa psycinfo'],
    'PubMed/MEDLINE': ['medline', 'pubmed', 'ncbi pubmed'],
    'RSWBplus': ['rswbplus'],
    #'Russian Science Citation Index': ['russian science citation index'],
    'ScienceOpen': ['scienceopen'],
    'Scopus': ['scopus', 'elsevier scopus'],
    'Web of Science': ['web of science', 'wos', 'thomson reuters web of science', 'web of sciences', 'webofscience',
                       'science citation index',  'web of science sci',
                       'social sciences citation index', 'ssci', 'web of science ssci',
                       'arts & humanities citation index', 'ahci', 'web of science ahci',
                       'derwent innovations index', 'derwent', 'web of science derwent',
                       'russian science citation index',
                       'book citation index', 'bci'],
    'Zasshi Kiji Sakuin': ['zasshi kiji sakuin', 'japanese periodicals index'],
    'Zentralblatt MATH': ['zentralblatt math', 'zbmath'],
    'The Zoological Record': ['the zoological record'],
    'Academic Search': ['academic search', 'ebsco academic search'],
    'Aerospace & High Technology Database': ['aerospace & high technology database'],
    'African Journals OnLine (AJOL)': ['african journals online', 'ajol'],
    'AgeLine': ['ageline'],
    'AGRICOLA': ['agricola', 'agricultural online access'],
    'Airiti': ['airiti'],
    'Analytical Abstracts': ['analytical abstracts'],
    'Anthropological Index Online': ['anthropological index online'],
    'Anthropological Literature': ['anthropological literature'],
    'ArchiveGrid': ['archivegrid'],
    'ASCE Library': ['asce library'],
    'AULIMP': ['aulimp', 'air university library index to military periodicals'],
    'Biological Abstracts': ['biological abstracts', 'biosis'],
    'Cabells': ['cabells'],
    'Chemical Abstracts Service': ['chemical abstracts service', 'cas', 'scifinder'],
    'Chinese Social Sciences Citation Index': ['chinese social sciences citation index'],
    'CHBD': ['chbd', 'circumpolar health bibliographic database'],
    'Civil Engineering Database': ['civil engineering database'],
    'Cochrane Library': ['cochrane library', 'cochrane database of systematic reviews', 'cdsr'],
    'Current Contents': ['current contents'],
    'DSI': ['dsi', 'stuttgart database of scientific illustrators 1450–1950'],
    'Index Copernicus': ['index copernicus'],
    'Information Bridge': ['information bridge', 'department of energy scientific and technical information'],
    'Indian Citation Index': ['indian citation index'],
    'IARP': ['iarp'],
    'INSPIRE-HEP': ['inspire-hep', 'inspire'],
    'International Directory of Philosophy': ['international directory of philosophy'],
    'International Medieval Bibliography': ['international medieval bibliography'],
    'JournalSeek': ['journalseek'],
    'Jurn': ['jurn'],
    'L\'Année philologique': ["l'année philologique"],
    'LexisNexis': ['lexisnexis', 'nexis', 'lexis-nexis'],
    'Lingbuzz': ['lingbuzz'],
    'Linguamatics': ['linguamatics'],
    'Logeion': ['logeion'],
    #'Mendeley': ['mendeley'],
    'National Criminal Justice Reference Service': ['national criminal justice reference service'],
    'OpenGrey': ['opengrey'],
    'ORCID': ['orcid'],
    'Philosophy Documentation Center eCollection': ['philosophy documentation center ecollection'],
    'POIESIS': ['poiesis', 'philosophy online serials'],
    'Publons': ['publons'],
    'PubPsych': ['pubpsych'],
    "Readers' Guide to Periodical Literature": ['readers guide to periodical literature'],
    "Rock's Backpages": ['rocks backpages'],
    'SafetyLit': ['safetylit'],
    'Science.gov': ['science.gov'],
    #'Science Citation Index': ['science citation index',  'web of science sci'], #'sci',
    'Scientific Information Database (SID)': ['scientific information database', 'sid'],
    'SCIndeks': ['scindeks', 'serbian citation index'],
    'SNAC': ['snac', 'social networks and archival contexts'],
    #'Social Sciences Citation Index': ['social sciences citation index', 'ssci', 'web of science ssci'],
    'Socolar': ['socolar'],
    "Ulrich's Periodicals Director": ['ulrichs periodicals directory'],
    'VET-Bib': ['vet-bib', 'european vocational education and training literature'],
    'WestLaw': ['westlaw'],
    'WorldCat': ['worldcat', 'oclc worldcat'],
    'WorldWideScience': ['worldwidescience'],
    'Association for Computing Machinery Digital Library': ['acm digital library', 'acm'],
    'Analytical Sciences Digital Library': ['analytical sciences digital library'],
    'Bibliographie de civilisation médiévale': ['bibliographie de civilisation médiévale'],
    'BioOne': ['bioone'],
    'IEEE Xplore': ['ieee xplore', 'ieee digital library', 'ieee'],
    'IngentaConnect': ['ingentaconnect'],
    'JSTOR': ['jstor', 'jstor archive'],
    'OpenEdition.org': ['openedition', 'openedition.org'],
    'Project MUSE': ['project muse', 'muse'],
    'SciELO': ['scielo', 'scielo library'],
    'ScienceDirect': ['sciencedirect', 'elsevier sciencedirect'],
    'SpringerLink': ['springerlink', 'springer link', 'springer'],
    'Bioinformatic Harvester': ['bioinformatic harvester'],
    'citeULike': ['citeulike'],
    'ChemXSeer': ['chemxseer'],
    'citebase Search': ['citebase search'],
    'Chemisches Zentralblatt Structural Database': ['chemisches zentralblatt structural database'],
    'CogPrints': ['cogprints'],
    'The Collection of Computer Science Bibliographies': ['collection of computer science bibliographies'],
    'Current Index to Statistics': ['current index to statistics'],
    'Ei Compendex': ['ei compendex'],
    'GENESIS': ['genesis'],
    #'Global Health': ['global health'],
    'HCI Bibliography': ['hci bibliography'],
    'International Bibliography of Humanism and the Renaissance': ['international bibliography of humanism and the renaissance'],
    'Meteorological and Geoastrophysical Abstracts': ['meteorological and geoastrophysical abstracts'],
    'Microsoft Academic': ['microsoft academic', 'microsoft research'],
    'Philosophy Research Index': ['philosophy research index'],
    'POPLINE': ['popline'],
    'Questia: Online Research Library': ['questia', 'questia online research library'],
    'Science Accelerator': ['science accelerator'],
    'Sparrho': ['sparrho'],

    'PsycARTICLES': ['psycarticles', 'apa psycarticles'],
    'British Library': ['british library', 'bl document supply'],
    'LILACS': ['lilacs'],
    'EBSCOhost': ['ebscohost', 'ebsco'],
    'Taylor & Francis': ['taylor & francis online', 'taylor & francis'],
    'PubAg': ['pubag', 'usda pubag'],
    'Ovid': ['ovid', 'ovid medline', 'ovidsp'],
    'Karger': ['karger journals', 'karger'],
    'Royal Society of Chemistry': ['royal society of chemistry', 'rsc', 'rsc publishing'],
    'SAGE': ['sage journals', 'sage'],
    'Springer': ['springerlink', 'springer', 'springer-verlag'],
    'zbMATH': ['zentralblatt math', 'zbmath'],
    'IEEE Standards': ['ieee standards', 'ieee std'],
    #'Book Citation Index': ['book citation index', 'bci'],
    #'Derwent Innovations Index': ['derwent innovations index', 'derwent', 'web of science derwent'],
    'IEEE Transactions': ['ieee transactions', 'ieee transactions library'],
    'ClinicalTrials.gov': ['clinicaltrials.gov', 'nih clinical trials'],
    'HathiTrust': ['hathitrust', 'hathitrust digital library'],
    'Emcare': ['emcare', 'nursing and allied health emcare'],
    'LegalTrac': ['legaltrac', 'legal resources index'],
    'Europeana': ['europeana', 'europeana collections'],
    'Scilit': ['scilit', 'scilit search engine'],
    'Gale Academic OneFile': ['gale academic onefile', 'gale databases'],
    'Bibliothèque nationale de France': ['bibliothèque nationale de france', 'bnf', 'bnf gallica'],
    'Eldis': ['databases for international development', 'eldis'],
    'MedCarib': ['medcarib', 'caribbean health sciences literature'],
    'bioRxiv': ['biorxiv', 'cold spring harbor biorxiv'],
    'chemRxiv': ['chemrxiv', 'chemistry preprint server'],
    'SPIN': ['spin', 'searchable physics information notices'],
    #'Zotero': ['zotero', 'zotero references'],
    'Digital Commons': ['digital commons network', 'digital commons'],
    'VHL': ['vhl', 'virtual health library', 'bvs', 'biblioteca virtual en salud'],
    'OpenCitations’ COCI': ['opencitations coci', 'coci', 'open citations coci'],
    'African Index Medicus': ['african index medicus'],
    'J-Stage': ['j-stage', 'jstage']

}


# Flatten the mapping for lookup
database_synonyms = {synonym: main_name for main_name, synonyms in literature_databases_mapping.items() for synonym in synonyms}

# Function to extract and merge databases from text
def extract_and_merge_databases(text, synonyms_mapping):
    found_databases = set()
    for synonym, main_name in synonyms_mapping.items():
        pattern = r'\b' + re.escape(synonym) + r'\b'  # Ensure we match whole words only
        if re.search(pattern, text):
            found_databases.add(main_name)
    return list(found_databases)

# Extract and merge databases from title, abstract, and keywords
df['databases_in_title'] = df['cleaned_title'].apply(lambda x: extract_and_merge_databases(x, database_synonyms))
df['databases_in_abstract'] = df['cleaned_abstract'].apply(lambda x: extract_and_merge_databases(x, database_synonyms))
df['databases_in_keywords'] = df['cleaned_keywords'].apply(lambda x: extract_and_merge_databases(x, database_synonyms))

# Combine the databases found in title, abstract, and keywords
df['all_databases_mentioned'] = df.apply(lambda row: list(set(row['databases_in_title'] + row['databases_in_abstract'] + row['databases_in_keywords'])), axis=1)

# Remove lists from the output columns and replace empty lists with NaN
df['databases_in_title'] = df['databases_in_title'].apply(lambda x: ', '.join(x) if x else np.nan)
df['databases_in_abstract'] = df['databases_in_abstract'].apply(lambda x: ', '.join(x) if x else np.nan)
df['databases_in_keywords'] = df['databases_in_keywords'].apply(lambda x: ', '.join(x) if x else np.nan)
df['all_databases_mentioned'] = df['all_databases_mentioned'].apply(lambda x: ', '.join(x) if x else np.nan)

# replace all empty cell by np.nan in df file
df.replace('', np.nan, inplace=True)

# Save results to a CSV, Excel, or Pickle file
df.to_csv('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_and_databases_20241013_tiab.csv', index=False)
df.to_pickle('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_and_databases_20241013_tiab.pkl')

# Display the first few rows of the dataframe
df

<ipython-input-10-87e23f99f0a8>:248: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('', np.nan, inplace=True)


,OpenAlex DOI,OpenAlex ID,OpenAlex Title,OpenAlex Abstract,OpenAlex Publication Year,OpenAlex Publication Date,OpenAlex Authors,OpenAlex First Authors,OpenAlex Last Authors,OpenAlex Corresponding Authors,OpenAlex Institutions,OpenAlex Institutions Type,OpenAlex First Authors Institutions Countries,OpenAlex Last Authors Institutions Countries,OpenAlex Institutions Distinct Count,OpenAlex Countries,OpenAlex First Authors Countries,OpenAlex Last Authors Countries,OpenAlex Corresponding Authors Countries,OpenAlex Countries Distinct Count,OpenAlex Journal,is_in_doaj,OpenAlex Crossref Publisher,OpenAlex Crossref Subject,OpenAlex topics,OpenAlex Keywords,OpenAlex Mesh,OpenAlex Concepts,OpenAlex Sustainable Development Goals,Crossref Citation count,OpenAlex Cited by Count,OpenAlex Counts by Year,OpenAlex Publication Type,OpenAlex Indexed In,OpenAlex Language,OpenAlex Open Access,OpenAlex Open Access status,OpenAlex grants,Crossref Funders,Altmetric Score,Altmetric Read Count,Altmetric Image,Altmetric URL,apc_list_value,apc_list_currency,apc_list_value_usd,apc_list_provenance,apc_paid_value,apc_paid_currency,apc_paid_value_usd,apc_paid_provenance,number_of_authors,Citations 2024,Citations 2023,Citations 2022,Citations 2021,Citations 2020,Citations 2019,Citations 2018,Citations 2017,Citations 2016,Citations 2015,Citations 2014,Citations 2013,Citations 2012,OpenAlex Countries Full Name,OpenAlex Countries Region,OpenAlex Countries Income,OpenAlex First Authors Countries Full Name,OpenAlex First Authors Countries Region,OpenAlex First Authors Countries Income,OpenAlex Last Authors Countries Full Name,OpenAlex Last Authors Countries Region,OpenAlex Last Authors Countries Income,OpenAlex Corresponding Authors Countries Full Name,OpenAlex Corresponding Authors Countries Region,OpenAlex Corresponding Authors Countries Income,publication_year,cleaned_title,cleaned_abstract,cleaned_keywords,databases_in_title,databases_in_abstract,databases_in_keywords,all_databases_mentioned
0,https://doi.org/10.1007/s11192-009-0146-3,https://openalex.org/W2150220236,"Software survey: VOSviewer, a computer program...","We present VOSviewer, a freely available compu...",2009,2009-12-30,"Nees Jan van Eck, Ludo Waltman",Nees Jan van Eck,Ludo Waltman,NaN,Leiden University (NL),Leiden University (education),Leiden University (NL),Leiden University (NL),1,NL,NL,NL,NaN,1,Scientometrics,False,Springer Science and Business Media LLC,NaN,Information Visualization and Visual Data Mini...,"Information Visualization, Interactive Visuali...",NaN,"Computer science, Construct (python library), ...",NaN,9908,12322,"[{'year': 2024, 'cited_by_count': 2487}, {'yea...",article,"['crossref', 'pubmed']",en,True,hybrid,NaN,NaN,87.484,9223.0,https://badges.altmetric.com/?size=64&score=88...,https://www.altmetric.com/details.php?citation...,2290.0,EUR,2890.0,doaj,2290.0,EUR,2890.0,doaj,2,2487.0,3183.0,2441.0,1652.0,991.0,571.0,359.0,172.0,132.0,94.0,78.0,44.0,27.0,Netherlands,Europe,High income,Netherlands,Europe,High income,Netherlands,Europe,High income,NaN,NaN,NaN,2009,software survey vosviewer a computer program f...,we present vosviewer a freely available comput...,information visualization interactive visualiz...,NaN,NaN,NaN,NaN
1,https://doi.org/10.1177/1094428114562629,https://openalex.org/W3125707221,Bibliometric Methods in Management and Organiz...,We aim to develop a meaningful single-source r...,2014,2014-12-22,"Ivan Župič, Tomaž Čater",Ivan Župič,Tomaž Čater,Ivan Župič,University of Ljubljana (SI),University of Ljubljana (education),University of Ljubljana (SI),University of Ljubljana (SI),1,SI,SI,SI,SI,1,Organizational Research Methods,False,SAGE Publications,NaN,Knowledge Management and Organizational Innova...,"Bibliographic coupling, Bibliometric Analysis,...",NaN,"Bibliographic coupling, Citation, Bibliometric...",Quality education,3038,3985,"[{'year': 2024, 'cited_by_count': 806}, {'year...",article,['crossref'],en,True,green,NaN,NaN,19.600,4730.0,https://badges.altme

In [ ]:
# List of literature databases with main names and synonyms grouped
literature_databases_mapping = {
    'PubMed': ['pubmed', 'medline', 'ncbi pubmed'],
    'Web of Science': ['web of science', 'web of sciences', 'wos', 'thomson reuters web of science','webofscience'],
    'Scopus': ['scopus', 'elsevier scopus'],
    'Embase': ['embase'],
    'Cochrane Library': ['cochrane library', 'cochrane database of systematic reviews', 'cdsr'],
    'Google Scholar': ['google scholar', 'scholar google'],
    'IEEE Xplore': ['ieee xplore', 'ieee digital library', 'ieee'],
    'ScienceDirect': ['sciencedirect', 'elsevier sciencedirect'],
    'JSTOR': ['jstor', 'jstor archive'],
    'PsycINFO': ['psycinfo', 'apa psycinfo'],
    'ERIC': ['eric', 'eric database'],
    'AGRIS': ['agris', 'agris international'],
    'CINAHL': ['cinahl', 'cinahl plus'],
    'BIOSIS': ['biosis previews', 'biosis', 'biological abstracts'],
    'ProQuest': ['proquest', 'proquest dissertation and theses', 'proquest central'],
    'Chemical Abstracts Service': ['cas', 'chemical abstracts service', 'scifinder'],
    'CAB Abstracts': ['cab abstracts', 'cabi'],
    'Inspec': ['inspec', 'iet inspec'],
    'ACM Digital Library': ['acm digital library', 'acm'],
    'LexisNexis': ['lexisnexis', 'nexis', 'lexis-nexis'],
    'PsycARTICLES': ['psycarticles', 'apa psycarticles'],
    'British Library': ['british library', 'bl document supply'],
    'LILACS': ['lilacs'],
    'EBSCOhost': ['ebscohost', 'ebsco'],
    'WorldCat': ['worldcat', 'oclc worldcat'],
    'arXiv': ['arxiv', 'cornell arxiv', 'arxiv.org'],
    'SSRN': ['ssrn', 'social science research network', 'ssrn elibrary'],
    'RePEc': ['repec', 'econpapers'],
    'Dimensions': ['dimensions', 'digital science dimensions'],
    'Wiley': ['wiley online library', 'wiley'],
    'Taylor & Francis': ['taylor & francis online', 'taylor & francis'],
    'PubAg': ['pubag', 'usda pubag'],
    'Ovid': ['ovid', 'ovid medline', 'ovidsp'],
    'Karger': ['karger journals', 'karger'],
    'Royal Society of Chemistry': ['royal society of chemistry', 'rsc', 'rsc publishing'],
    'SAGE': ['sage journals', 'sage'],
    'Springer': ['springerlink', 'springer', 'springer-verlag'],
    'SciELO': ['scielo', 'scielo library'],
    'CNKI': ['cnki', 'cnki scholar'],
    'zbMATH': ['zentralblatt math', 'zbmath'],
    'MathSciNet': ['mathscinet', 'ams mathscinet'],
    'EconLit': ['econlit', 'aea econlit'],
    'IEEE Standards': ['ieee standards', 'ieee std'],
    'OSTI': ['osti', 'osti.gov'],
    'NASA ADS': ['nasa ads', 'astrophysics data system'],
    'Europe PMC': ['europe pmc', 'pubmed central europe'],
    'The Lens': ['the lens', 'patent lens', 'lens.org'],
    'Social Sciences Citation Index': ['social sciences citation index', 'ssci', 'web of science ssci'],
    'Book Citation Index': ['book citation index', 'bci'],
    'Derwent Innovations Index': ['derwent innovations index', 'derwent', 'web of science derwent'],
    'IEEE Transactions': ['ieee transactions', 'ieee transactions library'],
    'ClinicalTrials.gov': ['clinicaltrials.gov', 'nih clinical trials'],
    'OpenAIRE': ['openaire', 'openaire infrastructure'],
    'HathiTrust': ['hathitrust', 'hathitrust digital library'],
    'Emcare': ['emcare', 'nursing and allied health emcare'],
    'DOAJ': ['doaj', 'directory of open access journals'],
    'J-Gate': ['j-gate', 'informatics j-gate'],
    'LegalTrac': ['legaltrac', 'legal resources index'],
    'Project MUSE': ['project muse', 'muse'],
    'Europeana': ['europeana', 'europeana collections'],
    'Scilit': ['scilit', 'scilit search engine'],
    'Gale Academic OneFile': ['gale academic onefile', 'gale databases'],
    'Bibliothèque nationale de France': ['bibliothèque nationale de france', 'bnf', 'bnf gallica'],
    'Eldis': ['databases for international development', 'eldis'],
    'MedCarib': ['medcarib', 'caribbean health sciences literature'],
    'bioRxiv': ['biorxiv', 'cold spring harbor biorxiv'],
    'chemRxiv': ['chemrxiv', 'chemistry preprint server'],
    'SPIN': ['spin', 'searchable physics information notices'],
    'Zotero': ['zotero', 'zotero references'],
    'Digital Commons': ['digital commons network', 'digital commons'],
    'VHL': ['vhl', 'virtual health library', 'bvs', 'biblioteca virtual en salud'],
    'OpenAlex': ['openalex'],
    'CrossRef': ['crossref'],
    'DataCite': ['datacite', 'data cite'],
    'PubMed Central': ['pubmed central', 'pmc'],
    'ResearchGate': ['researchgate', 'research gate'],
    'SpringerLink': ['springerlink', 'springer link', 'springer'],
    'Microsoft Academic': ['microsoft academic', 'microsoft research'],
    'OpenCitations’ COCI': ['opencitations coci', 'coci', 'open citations coci'],
    'CiteSeerX': ['citeseerx', 'citeseer'],
    'AMiner': ['aminer', 'a miner'],
    'African Index Medicus': ['african index medicus'],
    'African Journals Online': ['african journals online', 'ajol'],
    'J-Stage': ['j-stage', 'jstage']
}